In [2]:
# 0314 v2 - 가까운 두 개 마커로 위치 파악
# 마커 개수 늘림 시야에 3개까지 잘 잡히는데 가까운 2개를 고르려다 보니까 어떤 마커 2개를 고르는지에 따라서 좌표가 다르게 나타남

In [1]:
import cv2
import cv2.aruco as aruco
import numpy as np

# 1) 카메라 내부 파라미터 
# ✅ 캘리브이션된 카메라 파라미터 로드
cameraMatrix = np.load('C:/cal_cam/calibration_matrix.npy')
distCoeffs = np.load('C:/cal_cam/distortion_coefficients.npy')



# 2) 마커 실제 한 변 길이 (단위 m)
marker_length = 0.16

# 3) ArUco 사전, 파라미터 생성

aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)

detector_params = aruco.DetectorParameters()  # 예전 방식


# 4) 월드 좌표에서 마커의 위치(0~11번)
marker_world_pos = {
    0: np.array([0.0,  0.0, 0.0]),
    1: np.array([1.5,  0.0, 0.0]),
    2: np.array([3.0,  0.0, 0.0]),
    3: np.array([4.5,  0.0, 0.0]),
    4: np.array([0.0,  1.5, 0.0]),
    5: np.array([1.5,  1.5, 0.0]),
    6: np.array([3.0,  1.5, 0.0]),
    7: np.array([4.5,  1.5, 0.0]),
    8: np.array([0.0,  3.0, 0.0]),
    9: np.array([1.5,  3.0, 0.0]),
    10: np.array([3.0,  3.0, 0.0]),
    11: np.array([4.5,  3.0, 0.0]),
    12: np.array([0.7,  0.7, 0.0]),
    13: np.array([2.3,  0.7, 0.0]),
    14: np.array([3.7,  0.7, 0.0]),
    15: np.array([0.7,  2.3, 0.0]),
    16: np.array([2.3,  2.3, 0.0]),
    17: np.array([3.7,  2.3, 0.0]),
    
}

def compute_camera_pos(marker_id, rvec, tvec, marker_world):
    """
    단일 마커의 pose (rvec, tvec) + 해당 마커의 월드 좌표 -> 카메라 월드 위치 추정
    """
    R, _ = cv2.Rodrigues(rvec)        # (3,3)
    t = tvec.reshape(3, 1)            # (3,1)
    cam_in_marker = -R.T @ t          # (3,1)
    return marker_world.reshape(3, 1) + cam_in_marker

# 5) 카메라 열기
cap = cv2.VideoCapture(1)
cap.set(cv2.CAP_PROP_EXPOSURE, -6)

if not cap.isOpened():
    print("카메라를 열 수 없습니다.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("프레임 읽기 실패!")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 6) ArUco 마커 검출
    corners, ids, rejected = aruco.detectMarkers(
        gray, aruco_dict, parameters=detector_params
    )

    if ids is not None:
        # 마커 그리기
        aruco.drawDetectedMarkers(frame, corners, ids)

        # 7) Pose 추정
        rvecs, tvecs, _ = aruco.estimatePoseSingleMarkers(
            corners, marker_length, cameraMatrix, distCoeffs
        )

        # 마커 ID와 위치 표시
        for i, id_ in enumerate(ids):
            c = corners[i][0][0]
            cv2.putText(frame, f"ID {id_[0]}", (int(c[0]), int(c[1] - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # -------------------
        # [핵심] 가까운 두 마커로 카메라 위치 추정
        # -------------------
        if len(ids) >= 2:
            found_ids = ids.flatten()
            cam_positions = []

            # ID와 (rvec, tvec) 매핑
            id_to_pose = {id_[0]: (rvecs[i], tvecs[i]) for i, id_ in enumerate(ids)}

            # 월드 좌표 등록된 마커만 필터링
            valid_markers = [(id_, id_to_pose[id_][1]) for id_ in found_ids if id_ in marker_world_pos]

            if len(valid_markers) >= 2:
                # 거리순 (가까운 순)으로 정렬
                valid_markers.sort(key=lambda x: np.linalg.norm(x[1]))

                # 가까운 두 개 선택
                selected_ids = [valid_markers[0][0], valid_markers[1][0]]
                print(f"[INFO] Selected markers for position estimation: {selected_ids}")

                # 카메라 위치 계산
                for m_id in selected_ids:
                    rvec, tvec = id_to_pose[m_id]
                    world_pos = marker_world_pos[m_id]
                    cam_pos = compute_camera_pos(m_id, rvec, tvec, world_pos)
                    cam_positions.append(cam_pos)

                # 두 위치 평균
                camera_world_pos = np.mean(cam_positions, axis=0)
                cx, cy, cz = camera_world_pos.ravel()
                print(f"[INFO] Camera world position estimated: ({cx:.2f}, {cy:.2f}, {cz:.2f})")
    # -------------------

    # 화면 표시
    cv2.imshow("Aruco Marker Tracking", frame)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC 누르면 종료
        break

# 종료
cap.release()
cv2.destroyAllWindows()


[INFO] Selected markers for position estimation: [17, 11]
[INFO] Camera world position estimated: (3.87, 3.39, 2.46)
[INFO] Selected markers for position estimation: [17, 11]
[INFO] Camera world position estimated: (3.88, 3.41, 2.52)
[INFO] Selected markers for position estimation: [17, 11]
[INFO] Camera world position estimated: (3.81, 3.44, 2.43)
[INFO] Selected markers for position estimation: [17, 11]
[INFO] Camera world position estimated: (3.80, 3.45, 2.50)
[INFO] Selected markers for position estimation: [17, 11]
[INFO] Camera world position estimated: (3.85, 3.32, 2.47)
[INFO] Selected markers for position estimation: [17, 11]
[INFO] Camera world position estimated: (3.78, 3.36, 2.47)
[INFO] Selected markers for position estimation: [17, 11]
[INFO] Camera world position estimated: (3.76, 3.35, 2.46)
[INFO] Selected markers for position estimation: [17, 11]
[INFO] Camera world position estimated: (3.82, 3.37, 2.45)
[INFO] Selected markers for position estimation: [17, 11]
[INFO]